In [ ]:
from test_configuration import plaid_client, firestore_client
from flask import jsonify
import json
from datetime import date, datetime, timedelta

from plaid import ApiException
from plaid.model.transactions_get_request import TransactionsGetRequest
from plaid.model.transactions_get_response import TransactionsGetResponse

from plaid.model.accounts_balance_get_request import AccountsBalanceGetRequest
from plaid.model.accounts_get_response import AccountsGetResponse
from plaid.model.account_balance import AccountBalance

from google.cloud.firestore import DocumentReference
from google.cloud.firestore import CollectionReference

from typing import Any, Dict, Union, List

In [ ]:
def transactions_get(access_token: str, start_date: date, end_date: date):
    try:
        request = TransactionsGetRequest(
            access_token=access_token,
            start_date=start_date,
            end_date=end_date,
        )
        response: TransactionsGetResponse = plaid_client.transactions_get(
            request
        )

        return response.to_dict()

    except ApiException as e:
        exceptions: dict = json.loads(e.body)
        return exceptions

In [ ]:
def get_access_tokens(uid: str) -> Dict[str, Any]:
    try:
        user_doc = firestore_client.collection('users').document(uid)
        secrets_ref: CollectionReference = user_doc.collection('secrets')
        plaid_secrets_ref = secrets_ref.document('plaid')
        plaid_secrets_dict = plaid_secrets_ref.get().to_dict()
        access_tokens = plaid_secrets_dict.get('access_tokens')

        return {'status': 200, 'acess_tokens': access_tokens}
    except Exception as e:
        return {'status': 404, 'error_message': str(e)}

In [ ]:
def update_transactions(uid: str, transactions: List[Dict[str, Any]]):
    try:
        user_doc = firestore_client.collection('users').document(uid)
        transactions_collection: CollectionReference = user_doc.collection(
            'transactions')

        updated = 0
        created = 0

        for transaction in transactions:
            transaction_id = transaction.get('transaction_id')
            transaction_doc = transactions_collection.document(transaction_id)
            transaction_snapshot = transaction_doc.get()
            pending = transaction_snapshot.get('pending')

            if transaction_snapshot.exists and not pending:
                continue

            old_date: date = transaction.get('date')
            new_date = datetime(old_date.year, old_date.month, old_date.day)

            authorized_date: Union[date, None] = transaction.get(
                'authorized_date')

            if authorized_date is not None:
                authorized_date = datetime(
                    authorized_date.year, authorized_date.month, authorized_date.day)

            transaction.update({
                'date': new_date,
                'authorized_date': authorized_date,
            })

            if transaction_snapshot.exists:
                transaction_doc.update(transaction)
                updated += 1
            else:
                transaction_doc.create(transaction)
                created += 1

        return {'status': 200, 'message': f'successfully created {created} and updated {updated} transaction(s)'}
    except Exception as e:
        return {'status': 404, 'message': str(e)}

In [ ]:
def accounts_balance_get(access_token: str):
    try:
        request = AccountsBalanceGetRequest(access_token)
        response: AccountsGetResponse = plaid_client.accounts_balance_get(request)

        return response.to_dict()
    except ApiException as e:
        exceptions: dict = json.loads(e.body)
        return exceptions

In [ ]:
def update_accounts(uid: str, institution_id: str, accounts: Union[List[Dict], None]) -> Dict[str, Any]:
    
    try:
        user_doc = firestore_client.collection('users').document(uid)
        accounts_collections: CollectionReference = user_doc.collection('accounts')
        
        if accounts is None:
            accounts_query = accounts_collections.where('institution_id', '==', institution_id)
            accounts_stream = accounts_query.stream()

            for account in accounts_stream:
                account_doc = account.to_dict()
                account_id = account_doc.get('account_id')
                account_ref = accounts_collections.document(account_id)
                data = {
                    'account_connection_state': 'error',
                }

                account_ref.update(data)
        else:
            # Update Accounts in `Accounts` collections
            for account in accounts:

                # Update Accounts Collections
                account_id: str = account.get('account_id')
                print(f'accountId: {account_id}')

                account_doc_ref = accounts_collections.document(account_id)
                account_snapshot = account_doc_ref.get()

                balances: Union[AccountBalance, None] = account.get('balances')

                data = {
                    'account_id': account.get('account_id'),
                    'balances': {
                        'available': balances.get('available'),
                        'current': balances.get('current'),
                        'iso_currency_code': balances.get('iso_currency_code'),
                        'limit': balances.get('limit'),
                        'unofficial_currency_code': balances.get('unofficial_currency_code'),
                    },
                    'mask': account.get('mask'),
                    'name': account.get('name'),
                    'official_name': account.get('official_name'),
                    'subtype': str(account.get('subtype')),
                    'type': str(account.get('type')),
                    'verification_status': account.get('verification_status'),
                    'account_last_synced_time': datetime.now(),
                    'institution_id': institution_id,
                    'account_connection_state': 'healthy'
                }

                if account_snapshot.exists:
                    account_doc_ref.update(data)
                else:
                    account_doc_ref.set(data)

                result = {
                    'status': 200,
                    'message': 'successfully updated accounts data',
                }

        return result
    except Exception as e:
        result = {
            'status': 404,
            'error_message': str(e),
        }

        return result

In [ ]:
def transactions_refresh(request: Dict[str, Any]):
    # TODO: ^change the type of request to `flask.Request`

    # # TODO: uncomment below for release
    # # Parsing data from request to get `uid` and `public_token`
    # data_dict: dict = json.loads(request.data)
    # uid: Union[str, None] = data_dict.get('uid')   

    uid: Union[str, None] = request.get('uid') # for sandbox
    now = datetime.now()
    end_date = now.date()
    start_date = (now - timedelta(60)).date()

    # Get stored access_tokens
    access_tokens_response = get_access_tokens(uid)
    access_tokens: Union[Dict[str,str], None] = access_tokens_response.get('acess_tokens')
    print(f'access_tokens: {access_tokens}')

    if access_tokens is None:
        return access_tokens_response
    
    access_tokens_status = []
    
    for institution_id, access_token in access_tokens.items():
        print(f'code for {institution_id} started')

        update_status = {}
        update_status['institution_id'] = institution_id

        
        transacitons_get_response = transactions_get(
            access_token=access_token,
            start_date=start_date,
            end_date=end_date,
        )
        
        # Get Transactions data and update Firestore transactions collection
        transactions = transacitons_get_response.get('transactions')

        # If transaction is None, add the error
        if transactions is None:
            update_status['transactions_get_status'] = transacitons_get_response.get('error_code')
            update_status['transactions_get_message'] = transacitons_get_response.get('error_message')
        else:
            update_status['transactions_get_status'] = 200
            update_status['transactions_get_message'] = f'Successfully got {len(transactions)} transactions'

        update_transactions_result = update_transactions(uid, transactions)
        update_status['transaction_update_status'] = update_transactions_result.get('status')
        update_status['transaction_update_message'] = update_transactions_result.get('message')

        # Get /accounts/balance/get and update Firestore accounts collection
        balances_get_response = accounts_balance_get(access_token)
        accounts: Union[List[Dict],
                        None] = balances_get_response.get('accounts')

        # If transaction is None, add the error
        if accounts is None:
            update_status['accounts_get_status'] = balances_get_response.get('error_code')
            update_status['accounts_get_message'] = balances_get_response.get('error_message')
        else:
            update_status['accounts_get_status'] = 200
            update_status['accounts_get_message'] = f'Successfully got {len(accounts)} accounts'

        update_account_result = update_accounts(uid, institution_id, accounts)
        update_status['accounts_update_status'] = update_account_result.get('status')
        update_status['accounts_update_message'] = update_account_result.get('message')

        access_tokens_status.append(update_status)

    print(access_tokens_status)
    
    # TODO: uncomment below for release
    # return jsonify(status=200,access_tokens_status=access_tokens_status)

    return {'status':200,'access_tokens_status':access_tokens_status}

In [ ]:
data = {
    'uid': 'Io4KdAIWeXOO8EVGJVdP88lgYZC3',
}

transactions_refresh(data)